<a href="https://colab.research.google.com/github/mcmullrj/ME592_Public/blob/main/CombineControlEnvironment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def control_combine_environment(actions, logged_signals):
    # Global variables - environment defintion
    ##### redo this
    FieldPath = np.array([[0, 0], [1, 1]])  #read matrix
    CombineSettingNorm = 1  #read matrix
    FlowCropNorm = 1  #read matrix
    PowerGrainProcessNorm = 1  #read matrix
    GrainEfficiency = 1 #read matrix
    NormFuelEfficiencyVsPower = 1 #read matrix

    # Constants
    DurationTimeStep = 20  #sec
    SpeedCombineRef = 12  #km/hr
    TotalPowerRef = 399.4 #kW
    EnginePowerRef = 300 #kW
    FuelEfficiencyRef = 200 #g/hr/kW
    BatteryCapacity = 30 #kW-hr
    BatteryMaxChargeRate = 21 #kW
    BatteryMaxDischargeRate = 60 #kW
    MotorEfficiency = 0.9
    GrainPrice = 0.04
    FuelPrice = 1.0
    FlowCropRef = 16457.0 #### check this
    MaxPowerNormForPropulsion = 0.25;

    # Extracting actions and logged signals
    PowerEngineRequest, PowerMotorRequest, CombineSettingSetpoint = actions
    BatterySOCStartTimeStep, FieldIndexStartTimeStep = logged_signals

    # Limit inputs
    PowerEngine = max(0, min(PowerEngineRequest, EnginePowerRef))
    CombineSettingSetpoint = max(0.1, min(CombineSettingSetpoint, 1.7))

    # Adjust power inputs based on system state
    BatteryEnergy = BatterySOCStartTimeStep * BatteryCapacity
    PowerMotor = PowerMotorRequest

    # Physical limits for power requests
    if PowerMotor > BatteryMaxDischargeRate * MotorEfficiency:
      PowerMotor = BatteryMaxDischargeRate * MotorEfficiency
    elif PowerMotor < -BatteryMaxChargeRate:
      PowerMotor = -BatteryMaxChargeRate

    # Normalized power
    PowerNorm = (PowerEngine + PowerMotor) / TotalPowerRef

    # Overspecified power: meet requested engine power, reduce motor power to get total = max
    if PowerNorm > 1:
        PowerNorm = 1
        PowerMotor = PowerNorm * TotalPowerRef - PowerEngine

    #create combine speed array for interpolation
    ###### check this
    SpeedCombineInterp = np.linspace(0.0,1.0,50)*SpeedCombineRef*1000/3600 #m/sec

    # Calculate FieldMapGridDistance in meters per grid
    FieldMapGridDistance = FieldPath[1, 0] - FieldPath[0, 0]

    # Calculate GridsPerTimeStepInterp
    GridsPerTimeStepInterp = (SpeedCombineInterp / FieldMapGridDistance) * DurationTimeStep

    # Calculate FieldIndexEndTimeStepInterp
    FieldIndexEndTimeStepInterp = FieldIndexStartTimeStep + round(GridsPerTimeStepInterp[-1])

    # Calculate TimePerGrid in seconds
    TimePerGrid = FieldMapGridDistance / SpeedCombineInterp
    TimePerGrid[0] = 2 * TimePerGrid[1]

    # Calculate CropRateNormInterp
    CropRateNormInterp = (FieldPath[FieldIndexStartTimeStep:FieldIndexEndTimeStepInterp, 1] * (1. / TimePerGrid)) * (3600. / FlowCropRef)

    # Interpolate PowerNormGrainHarvest
    PowerNormGrainHarvestInterp = np.interp(CombineSettingNorm, FlowCropNorm, PowerGrainProcessNorm, CombineSettingSetpoint * np.ones_like(CropRateNormInterp), CropRateNormInterp)

    # Solve for speed from power
    BatterySOCInterp = np.ones_like(PowerNormGrainHarvestInterp) * BatterySOCStartTimeStep
    BatteryEnergy = np.ones_like(SpeedCombineInterp) * BatteryEnergy
    #
    PowerNormForPropulsion = PowerNorm - PowerNormGrainHarvestInterp
    PowerEngineInterp = np.ones_like(PowerNormGrainHarvestInterp) * PowerEngine
    PowerMotorInterp = np.ones_like(PowerNormGrainHarvestInterp) * PowerMotor
    PowerEngine = PowerEngineInterp[0]
    PowerMotor = PowerMotorInterp[0]

    for k1 in range(PowerNormForPropulsion.shape[0]):
      # Update norm power if battery depleted (discharging) or fully charged (charging)
      BatteryEnergy = BatteryEnergy - PowerMotor * (TimePerGrid / 3600)  # kWh
      BatterySOCInterp[k1, :] = BatteryEnergy / BatteryCapacity

      for k2 in range(PowerNormForPropulsion.shape[1]):
        if BatterySOCInterp[k1, k2] < 0.1 and PowerMotor[k2] >= 0:  # Discharging
            PowerMotor[k2] = 0
            PowerMotorInterp[k1, k2] = 0
            PowerNormForPropulsion[k1, k2] = PowerEngine[k2] / TotalPowerRef - PowerNormGrainHarvestInterp[k1, k2]
        elif BatterySOCInterp[k1, k2] > 0.9 and PowerMotor[k2] <= 0:  # Charging
            PowerMotor[k2] = 0
            PowerEngine[k2] = PowerEngine[k2] - PowerMotor[k2]  # All engine power goes to combine
            PowerMotorInterp[k1, k2] = 0
            PowerEngineInterp[k1, k2] = PowerEngine[k2]
            PowerNormForPropulsion[k1, k2] = PowerEngine[k2] / TotalPowerRef - PowerNormGrainHarvestInterp[k1, k2]

      # Check if any power left for propulsion
      for k2 in range(PowerNormForPropulsion.shape[1]):
        if PowerNormForPropulsion[k1, k2] < 0:
            PowerNormForPropulsion[k1, k2] = 0  # 0 power for propulsion is the minimum possible

    SpeedFromPower = (PowerNormForPropulsion / MaxPowerNormForPropulsion)**0.5 * (SpeedCombineRef * 1000 / 3600)

    # Calculate error and pick speed for min error
    Error2SpeedCombine = ((np.ones((SpeedFromPower.shape[0], 1)) @ SpeedCombineInterp - SpeedFromPower) ** 2)

    # Find min error for each row in potential solution space
    SpeedCombineGrid = np.zeros(SpeedFromPower.shape[0])
    CropRateNormGrid = np.zeros(SpeedFromPower.shape[0])
    PowerEngineGrid = np.ones(SpeedFromPower.shape[0])  # kW
    PowerMotorGrid = np.ones(SpeedFromPower.shape[0])  # kW
    BatterySOCGrid = np.zeros(SpeedFromPower.shape[0])
    TimeGrid = np.zeros(SpeedFromPower.shape[0])

    for k1 in range(Error2SpeedCombine.shape[0]):
      MinError2SpeedCombine = np.min(Error2SpeedCombine[k1, :])
      IndexSpeedSolution = np.argmin(Error2SpeedCombine[k1, :])

      if IndexSpeedSolution == Error2SpeedCombine.shape[1] - 1:
          # Combine is at its governed speed limit
          PowerNormGrid = MaxPowerNormForPropulsion + PowerNormGrainHarvestInterp[k1, -1]
          PowerEngineGrid[k1] = PowerNormGrid * TotalPowerRef - PowerMotorInterp[k1, -1]
      else:
          PowerEngineGrid[k1] = PowerEngineInterp[k1, IndexSpeedSolution]

      # Select answers for crop volume and combine speed
      TimeGrid[k1] = TimePerGrid[IndexSpeedSolution]
      SpeedCombineGrid[k1] = SpeedCombineInterp[IndexSpeedSolution] / 1000 * 3600  # km/hr
      CropRateNormGrid[k1] = CropRateNormInterp[k1, IndexSpeedSolution]
      PowerMotorGrid[k1] = PowerMotorInterp[k1, IndexSpeedSolution]
      BatterySOCGrid[k1] = BatterySOCInterp[k1, IndexSpeedSolution]

    # Calculate grids harvested in time step
    TimeCumulative = 0
    Grid = 1
    while TimeCumulative < DurationTimeStep:
      # Calculate TimeGrid for the current grid
      # TimeGrid = FieldMapGridDistance / (SpeedCombineGrid[Grid] * 1000 / 3600)  # sec
      TimeCumulative += TimeGrid[Grid]
      Grid += 1

    if Grid > Error2SpeedCombine.shape[1]:
      Grid = Error2SpeedCombine.shape[1]

    FieldIndexEndTimeStep = FieldIndexStartTimeStep + Grid - 1
    SpeedCombineGrid = SpeedCombineGrid[:Grid]
    CropRateNormGrid = CropRateNormGrid[:Grid]
    TimeGrid = TimeGrid[:Grid]
    TimeCumulative = np.sum(TimeGrid)
    PowerEngineGrid = PowerEngineGrid[:Grid]
    PowerMotorGrid = PowerMotorGrid[:Grid]

    # Interpolate Grain Efficiency
    EfficiencyGrainHarvestGrid = np.interp(CombineSettingNorm, FlowCropNorm, GrainEfficiency, CombineSettingSetpoint * np.ones_like(CropRateNormGrid), CropRateNormGrid)

    # Calculate Grain Harvest per Grid
    GrainPerGrid = FieldPath[FieldIndexStartTimeStep:FieldIndexEndTimeStep, 2] * EfficiencyGrainHarvestGrid

    # Calculate Specific Fuel Consumption Grid
    SpecificFuelConsumptionGrid = np.interp(PowerEngineGrid / EnginePowerRef, NormFuelEfficiencyVsPower[:, 0], NormFuelEfficiencyVsPower[:, 1], left=np.nan, right=np.nan) * FuelEfficiencyRef  # g/kW/hr

    # Calculate Fuel Rate Grid
    FuelRateGrid = SpecificFuelConsumptionGrid * PowerEngineGrid / (0.82 * 3.785 * 1000)  # gal/hr

    # Calculate Fuel Consumption Grid
    FuelConsumptionGrid = FuelRateGrid * (TimeGrid / 3600)  # gal

    # Calculate Grain Harvested, Fuel Consumed, Grain Harvest Value, and Fuel Cost
    GrainHarvested = np.sum(GrainPerGrid)  # bu in time step
    FuelConsumed = np.sum(FuelConsumptionGrid)  # gal in time step
    GrainHarvestValue = GrainHarvested * GrainPrice  # $
    FuelCost = FuelConsumed * FuelPrice  # $

    # Calculate SpeedCombine and EfficiencyGrainHarvest
    SpeedCombine = np.sum(SpeedCombineGrid * (TimeGrid / TimeCumulative))
    EfficiencyGrainHarvest = np.sum(EfficiencyGrainHarvestGrid * (TimeGrid / TimeCumulative))

    # Calculate BatterySOC
    BatterySOC = BatterySOCGrid[-1]

    # Calculate StateVector
    state_vector = [SpeedCombine, EfficiencyGrainHarvest, BatterySOC]

    # Calculate Reward
    reward = GrainHarvestValue - FuelCost

    # Update LoggedSignals.Diagnostics
    # LoggedSignals.Diagnostics = [PowerEngineMean, PowerMotorMean, CropRateNorm, FuelRate]

    # Calculate FieldIndexStartNextTimeStep
    FieldIndexStartNextTimeStep = FieldIndexEndTimeStep + 1

    # Update LoggedSignals.StartTimeStep
    logged_signals = [BatterySOC, FieldIndexStartNextTimeStep]

    if len(FieldPath[:, 0]) - np.max(GridsPerTimeStepInterp) < FieldIndexStartNextTimeStep:
        is_done = 1
    else:
        is_done = 0

    return state_vector, reward, is_done, logged_signals


SyntaxError: invalid syntax (<ipython-input-2-b0765afb1e80>, line 203)